In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.9MB/s 
     |████████████████████████████████| 3.3MB 25.0MB/s 
     |████████████████████████████████| 901kB 40.1MB/s 


In [3]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
  
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")


KeyboardInterrupt: ignored

In [2]:
text = "Сегодня пасмурно?"
tokenized_text = tokenizer.tokenize(text)
#print(tokenizer.encode())
for i, item in enumerate(tokenized_text):
  changed_token = tokenized_text[i]
  if changed_token == '▁':
    continue
  if changed_token.startswith('▁'):  
    #print(changed_token)
    changed_token = changed_token[1:]
  masked = tokenized_text[0:i]
  masked.append(tokenizer.mask_token) 
  masked += tokenized_text[i+1:]
  masked_text = tokenizer.convert_tokens_to_string(masked)
  input = tokenizer.encode_plus(masked_text, return_tensors = "pt")
  mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
  output = model(**input)
  logits = output.logits
  softmax = F.softmax(logits, dim = -1)
  mask_word = softmax[0, mask_index, :] 
  values_10, indices_10 = torch.topk(mask_word, 10, dim = 1)
  predicted_words_10 =[tokenizer.decode(token) for token in indices_10[0]] 

  if changed_token in predicted_words_10:
    print("Токен верен")
  else:
    print(changed_token, predicted_words_10)



  #print(masked_text, changed_token)
print('____________________________-')



NameError: ignored

In [4]:
from transformers import AutoTokenizer, BertModel
  
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

encoder = BertModel.from_pretrained("DeepPavlov/rubert-base-cased" )

In [5]:
import numpy as np


In [10]:

# Загрузка необходимых библиотек и модулей

import zipfile # Библиотека для работы с zip архивами
import os      # Библиотека для работы с фаловой системой 
import time    # Библиотека для работы со временем

from google.colab import drive # Модуль для работы с Google Dis
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
SRC_TEXT = './drive/MyDrive/ML/src_text50000.txt'
TGT_TEXT = './drive/MyDrive/ML/tgt_text50000.txt'

In [12]:
def download_dataset(path, skip_news_splitter = False):
  dataset = []
  lines = []
  line_counter = 0
  rec = []
  with open(path, 'r') as file:
    lines = file.readlines()

  for i, line in enumerate(lines):
    if line != '\n':
      if line.endswith('\n'):
        line = line[0: -1]
      if skip_news_splitter:
        dataset.append(line)
      else:
        rec.append(line)
      line_counter += 1
    else: 
      if not skip_news_splitter:
        dataset.append(rec)
        rec = []
  return dataset, line_counter

In [13]:
src_texts, size = download_dataset(SRC_TEXT, True)
print(size)
tgt_texts, size = download_dataset(TGT_TEXT, True)
print(size)

411085
411085


In [20]:
src = src_texts[8]
tgt = tgt_texts[8]
tok_src = tokenizer.encode_plus(src, return_tensors="pt")
tok_tgt = tokenizer.encode_plus(tgt, return_tensors="pt")
tok_src = np.array(tok_src.tokens())
tok_tgt = np.array(tok_tgt.tokens())
print(tok_src)
print(get_mask_wordstart(tok_src))
print(tok_tgt)
print(get_mask_wordstart(tok_tgt))

['[CLS]' 'за' 'задержание' 'отваж' '##ного' 'летчика' 'была' 'объявлена'
 'объявлена' 'большая' 'премия' '.' '[SEP]']
[False, True, True, True, False, True, True, True, True, True, True, True, False]
['[CLS]' 'за' 'задержание' 'отваж' '##ного' 'летчика' 'была' 'объявлена'
 'большая' 'премия' '.' '[SEP]']
[False, True, True, True, False, True, True, True, True, True, True, False]


In [6]:
def get_mask_wordstart(tokens):
  masks = []

  for i, token in enumerate(tokens):
    masks.append(not token.startswith('##') and token not in tokenizer.special_tokens_map.values())

  return masks

In [7]:

text = "Я в универе"
input = tokenizer.encode_plus(text, return_tensors="pt")
nice = np.array(input.tokens())

mask = get_mask_wordstart(nice)
print(mask)
#print(nice[mask] )
output = encoder(**input)
print(output.last_hidden_state[0][mask])

[False, True, True, True, False, False]
tensor([[ 0.4377, -0.4843, -0.1764,  ..., -0.3120, -0.6090, -0.6357],
        [ 1.2454, -0.4027, -0.0221,  ...,  0.4667, -0.0821, -0.4957],
        [ 0.4238, -0.3693,  0.0766,  ..., -0.7490,  0.2599, -0.3624]],
       grad_fn=<IndexBackward>)


In [8]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
import torch.nn as nn
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=10):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
decoder1 = DecoderRNN(hidden_size, tokenizer.vocab_size).to(device)
trainIters(encoder, decoder1, 75000, print_every=5000)

119547


NameError: ignored